# Network Dynamics and Graph - Politecnico di Torino - Hafez Ghaemi - S289963
## HW1 - Exercise 3

In [2]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import cvxpy as cp

## Reading the data and creating the graph

In [2]:
import scipy.io
f = scipy.io.loadmat('flow.mat')["flow"].reshape(28,)
C = scipy.io.loadmat('capacities.mat')["capacities"].reshape(28,)
B = scipy.io.loadmat('traffic.mat')["traffic"]
l = scipy.io.loadmat('traveltime.mat')["traveltime"].reshape(28,)


In [3]:
nV = np.shape(B)[0]
nE = np.shape(B)[1]

In [5]:
G = nx.DiGraph()
for e in range(nE):
  G.add_edge(list(B[:,e]).index(1)+1, list(B[:,e]).index(-1)+1, weight=l[e], capacity = C[e])

## Part a

In [6]:
print("The shortest path and its total cost:")
print(nx.shortest_path(G,1,17))
print(nx.shortest_path_length(G,1,17, weight="weight"))


The shortest path and its total cost:
[1, 2, 3, 9, 13, 17]
0.532996


## Part b

## The maximum flow

In [7]:
print("The maximum flow and its path:")
print(nx.maximum_flow(G,1,17)[0])
print(nx.maximum_flow(G,1,17)[1])

The maximum flow and its path:
22448
{1: {2: 8741, 6: 13707}, 2: {3: 8741, 7: 0}, 3: {4: 0, 8: 0, 9: 8741}, 4: {5: 0, 9: 0}, 5: {14: 0}, 6: {7: 4624, 10: 9083}, 7: {8: 4624, 10: 0}, 8: {9: 4624, 11: 0}, 9: {13: 6297, 12: 7068}, 13: {14: 3835, 17: 10355}, 14: {17: 3835}, 10: {11: 825, 15: 8258}, 11: {12: 825, 15: 0}, 15: {16: 8258}, 12: {13: 7893}, 17: {}, 16: {17: 8258}}


## Part c

In [8]:
v = B@f
v1 = v[0]
v17 = -v1

In [9]:
print("The exteranl flows satisfying v=Bf")
print(v)

The exteranl flows satisfying v=Bf
[ 16806   8570  19448   4957   -746   4768    413     -2  -5671   1169
     -5  -7131   -380  -7412  -7810  -3430 -23544]


In [14]:
v_new = np.zeros(nV)
v_new[0] = v1
v_new[16] = v17

## Part d

In [15]:
def totalDelay(f):
    return np.sum(f*l/(1-f/C))

In [16]:
f_var = cp.Variable(nE)
SO_obje = cp.Minimize(cp.sum(cp.multiply(cp.multiply(l,C), cp.inv_pos(1-(cp.multiply(f_var, cp.inv_pos(C)))))-cp.multiply(l,C)))
SO_cons = [B@f_var==v_new, f_var>=0, f_var<=C]
prob = cp.Problem(SO_obje,SO_cons)
prob.solve()
SO_flow = f_var.value
SO_delay = totalDelay(SO_flow)

In [17]:
print("Total delay in social optimum: ", SO_delay.round(1))
print("Flows in social optimum: ", SO_flow.round(1))

Total delay in social optimum:  25943.6
Flows in social optimum:  [ 6642.2  6058.9  3132.3  3132.3 10163.8  4638.3  3006.3  2542.6  3131.5
   583.3     0.   2926.6     0.   3132.3  5525.5  2854.3  4886.4  2215.2
   463.7  2337.7  3318.   5655.7  2373.1     0.   6414.1  5505.4  4886.5
  4886.5]


## Part e1

In [21]:
f_var = cp.Variable(nE)
Ward_obje = cp.Minimize(cp.sum(-cp.multiply(cp.multiply(l,C),cp.log(C-f_var)-cp.log(C))))
Ward_cons = [B@f_var==v_new, f_var>=0, f_var<=C]
prob = cp.Problem(Ward_obje,Ward_cons)
prob.solve()
Ward_flow = f_var.value
Ward_delay = totalDelay(Ward_flow)


In [22]:
print("Total delay in Wardrope without tolls: ", Ward_delay.round(1))
print("Flows in Wardrope without tolls: ", Ward_flow.round(1))

Total delay in Wardrope without tolls:  26293.0
Flows in Wardrope without tolls:  [ 6715.6  6715.6  2367.4  2367.4 10090.4  4645.4  2803.8  2283.6  3418.5
     0.    176.8  4171.4     0.   2367.4  5445.   2353.2  4933.3  1841.6
   697.1  3036.5  3050.3  6086.8  2586.5     0.   6918.7  4953.9  4933.3
  4933.3]


In [23]:
poa = Ward_delay/SO_delay
print("The price of anarchy without tolls:", poa)

The price of anarchy without tolls: 1.0134653783719794


## Part e2

In [24]:
f_var = cp.Variable(nE)
omega = SO_flow*C*l/(SO_flow-C)**2
Ward_toll_obje = cp.Minimize(cp.sum(cp.multiply(cp.multiply(l,C),cp.log(C)-cp.log(C-f_var)) + cp.multiply(f_var,omega)))
Ward_toll_cons = [B@f_var==v_new, f_var>=0, f_var<=C]
prob = cp.Problem(Ward_toll_obje,Ward_toll_cons)
prob.solve()
prob.solve()
Ward_toll_flow = f_var.value
Ward_toll_delay = totalDelay(Ward_toll_flow)

In [25]:
print("Total delay in Wardrope with tolls: ", Ward_toll_delay.round(1))
print("Flows in Wardrope with tolls: ", Ward_toll_flow.round(1))

Total delay in Wardrope with tolls:  25943.6
Flows in Wardrope with tolls:  [ 6643.   6059.1  3132.5  3132.5 10163.   4638.3  3006.3  2542.3  3131.5
   583.9     0.   2926.6     0.   3132.5  5524.8  2854.2  4886.4  2215.8
   464.   2337.5  3318.2  5655.7  2373.      0.   6414.1  5505.5  4886.4
  4886.4]


In [26]:
poa = Ward_toll_delay/SO_delay
print("The price of anarchy with tolls:", round(poa,3))

The price of anarchy with tolls: 1.0


## Part f

In [27]:
def tot_cost_new(f):
    return np.sum(f*l/(1-f/C)-f*l)

In [30]:
f_var = cp.Variable(nE)
SO_obje_new_cost = cp.Minimize(cp.sum(cp.multiply(cp.multiply(l,C), cp.inv_pos(1-(cp.multiply(f_var, cp.inv_pos(C)))))- cp.multiply(l,C)- cp.multiply(f_var, l)))
SO_cons_new_cost = [B@f_var==v_new, f_var>=0, f_var<=C]
prob = cp.Problem(SO_obje_new_cost,SO_cons_new_cost)
prob.solve()
SO_flow_new_cost_flow = f_var.value
SO_new_cost = tot_cost_new(SO_flow_new_cost_flow)

In [31]:
print("Total cost in scoial optimum with new cost function: ", SO_new_cost.round(1))
print("Flows in social optimum with the new cost function: ", SO_flow_new_cost_flow.round(1))

Total cost in scoial optimum with new cost function:  15095.5
Flows in social optimum with the new cost function:  [ 6653.3  5774.7  3419.7  3419.7 10152.7  4642.8  3105.8  2662.2  3009.1
   878.6     0.   2354.9     0.   3419.7  5509.9  3043.7  4881.8  2415.6
   443.7  2008.   3487.4  5495.4  2203.8     0.   6300.7  5623.5  4881.8
  4881.8]


In [33]:
omega_new = SO_flow_new_cost_flow*C*l/(SO_flow_new_cost_flow-C)**2
f_var = cp.Variable(nE)
Ward_toll_obje_new= cp.Minimize(cp.sum(cp.multiply(l,cp.multiply(C,cp.log(C))-cp.multiply(C,cp.log(C-f_var))-f_var)+cp.multiply(f_var,omega_new)))
Ward_toll_cons_new = [B@f_var==v_new, f_var>=0, f_var<=C]
prob = cp.Problem(Ward_toll_obje_new,Ward_toll_cons_new)
prob.solve()
Ward_toll_flow_new = f_var.value
Ward_toll_new_total_cost = tot_cost_new(Ward_toll_flow_new)

In [34]:
print("Total Wardrope cost with tolls with the new cost function: ", Ward_toll_new_total_cost.round(1))
print("Flows in Wardrope with tolls with the new cost function: ", Ward_toll_flow_new.round(1))

Total Wardrope cost with tolls with the new cost function:  15095.5
Flows in Wardrope with tolls with the new cost function:  [ 6653.1  5775.4  3419.5  3419.5 10152.9  4642.4  3105.5  2661.7  3009.2
   877.7     0.   2355.9     0.   3419.5  5510.4  3043.4  4881.7  2414.6
   443.8  2008.5  3487.1  5495.6  2204.1     0.   6300.7  5623.5  4881.7
  4881.7]
